In [27]:
import lxml.html as lh
import time
import urllib.request
import argparse
import datetime
import pytz
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import os
import coloredlogs

from docopt import docopt
from trading_bot.ops import get_state
from trading_bot.agent import Agent
from trading_bot.methods import evaluate_model
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)

In [28]:
tz = pytz.timezone('Asia/Kolkata')
#tz = pytz.timezone('Australia/Melbourne')
path1 = os.getcwd()
path = path1 + '/chromedriver'

ignored_exceptions=(StaleElementReferenceException,)
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(executable_path=path , options=options)

2020-05-22 09:20:07 matorix-Inspiron-15-3567 selenium.webdriver.remote.remote_connection[3412] DEBUG POST http://127.0.0.1:34037/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"excludeSwitches": ["enable-logging"], "extensions": [], "args": ["--no-sandbox", "--disable-dev-shm-usage", "--headless"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"excludeSwitches": ["enable-logging"], "extensions": [], "args": ["--no-sandbox", "--disable-dev-shm-usage", "--headless"]}}}
2020-05-22 09:20:07 matorix-Inspiron-15-3567 urllib3.connectionpool[3412] DEBUG Starting new HTTP connection (1): 127.0.0.1:34037
2020-05-22 09:20:18 matorix-Inspiron-15-3567 urllib3.connectionpool[3412] DEBUG http://127.0.0.1:34037 "POST /session HTTP/1.1" 200 679
2020-05-22 09:20:18 matorix-Inspiron-15-3567 selenium.webdriver.remote.remote_connection[3412] DEBUG Finished Req

In [29]:
#This function returns the table of the given url
def Real(url,count):
    if count == 0:
        
        driver.get(url)
        #print(driver)

    else:
        driver.refresh()
        time.sleep(15)

    infile = driver.page_source
    doc = lh.fromstring(infile)
    live = doc.xpath('/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[4]/div/div/div/div[3]/div/div/span[1]')
    live = float(live[0].text.replace(',',''))
    return live 

In [30]:
def evaluate_model(agent, state, next_state, data, t, total_profit, history, reward, window_size, debug=False):
  
    print(t)
        # select an action
    action = agent.act(state, is_eval=True)

        # BUY
    if action == 1:
        agent.inventory.append(data[t])

        history.append((data[t], "BUY"))
        if debug:
            logging.debug("Buy at: {}".format(format_currency(data[t])))

        # SELL
    elif action == 2 and len(agent.inventory) > 0:
        bought_price = agent.inventory.pop(0)
        delta = data[t] - bought_price
        reward = delta #max(delta, 0)
        total_profit += delta

        history.append((data[t], "SELL"))
        if debug:
            logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
        # HOLD
    else:
        history.append((data[t], "HOLD"))

#        done = (t == data_length - 1)
    agent.memory.append((state, action, reward, next_state))

    return total_profit

In [31]:
# def main(args):
#     count = 0
#     total_profit = 0
#     t=0
#     history = []
#     reward = 0
#     ticker = args + '.NS'
#     price = []
#     window_size =10
#     time_now = datetime.datetime.now(tz).time()
#     while(datetime.time(9, 14, tzinfo=tz) < time_now < datetime.time(15, 31, tzinfo=tz)):
#         url = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'.format(ticker,ticker)
#         print(count)
#         live = Real(url,count)
#         count+=1        
#         price.append(live)
#         if count < window_size:
#            continue
#         model_name='model_debug_50'  
#         print(live)
#         initial_offset = price[1] - price[0]
#         state = get_state(price, 0, window_size + 1)
#         next_state = get_state(price, t + 1, window_size + 1)
#         agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
#         agent.inventory = []
#         profit = evaluate_model(agent,state,next_state, price, t, total_profit, history, reward, window_size=window_size)
#         show_eval_result(model_name, profit, initial_offset)
#         t+=1
#         state = next_state

In [32]:
#args = ticker('RELIANCE')
coloredlogs.install(level="DEBUG")
switch_k_backend_device()

try:
    #main('RELIANCE')
    args = 'RELIANCE'
    count = 0
    total_profit = 0
    t=0
    history = []
    reward = 0
    ticker = args + '.NS'
    price = []
    window_size =10
    time_now = datetime.datetime.now(tz).time()
    while(datetime.time(9, 14, tzinfo=tz) < time_now < datetime.time(15, 31, tzinfo=tz)):
        url = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'.format(ticker,ticker)
        print(count)
        live = Real(url,count)
        count+=1        
        price.append(live)
        if count < window_size:
           continue
        model_name='model_debug_50'  
        print(live)
        initial_offset = price[1] - price[0]
        state = get_state(price, 0, window_size + 1)
        next_state = get_state(price, t + 1, window_size + 1)
        agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
        agent.inventory = []
        profit = evaluate_model(agent,state,next_state, price, t, total_profit, history, reward, window_size=window_size)
        show_eval_result(model_name, profit, initial_offset)
        t+=1
        state = next_state
except KeyboardInterrupt:
    print("Aborted")  

12] DEBUG Finished Request
2020-05-22 09:49:04 matorix-Inspiron-15-3567 selenium.webdriver.remote.remote_connection[3412] DEBUG GET http://127.0.0.1:34037/session/066c19efb0de621e642353f3daf1c328/source {}
2020-05-22 09:49:04 matorix-Inspiron-15-3567 urllib3.connectionpool[3412] DEBUG http://127.0.0.1:34037 "GET /session/066c19efb0de621e642353f3daf1c328/source HTTP/1.1" 200 888079
2020-05-22 09:49:04 matorix-Inspiron-15-3567 selenium.webdriver.remote.remote_connection[3412] DEBUG Finished Request
1450.0
2020-05-22 09:49:05 matorix-Inspiron-15-3567 root[3412] INFO model_debug_50: USELESS

2020-05-22 09:49:05 matorix-Inspiron-15-3567 selenium.webdriver.remote.remote_connection[3412] DEBUG POST http://127.0.0.1:34037/session/066c19efb0de621e642353f3daf1c328/refresh {}
89
99
2020-05-22 09:49:06 matorix-Inspiron-15-3567 urllib3.connectionpool[3412] DEBUG http://127.0.0.1:34037 "POST /session/066c19efb0de621e642353f3daf1c328/refresh HTTP/1.1" 200 14
2020-05-22 09:49:06 matorix-Inspiron-15-35

In [19]:
# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument('ticker', help = 'ticker')
#     args = parser.parse_args()

#     coloredlogs.install(level="DEBUG")
#     switch_k_backend_device()

#     try:
#         main(args)
#     except KeyboardInterrupt:
#         print("Aborted")  


### Visualising Data

In [33]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

2020-05-22 09:55:05 matorix-Inspiron-15-3567 matplotlib[3412] DEBUG CACHEDIR=/home/matorix/.cache/matplotlib
2020-05-22 09:55:05 matorix-Inspiron-15-3567 matplotlib.font_manager[3412] DEBUG Using fontManager instance from /home/matorix/.cache/matplotlib/fontlist-v310.json
2020-05-22 09:55:07 matorix-Inspiron-15-3567 matplotlib.pyplot[3412] DEBUG Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2020-05-22 09:55:07 matorix-Inspiron-15-3567 matplotlib.pyplot[3412] DEBUG Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [ ]:
# from trading_bot.agent import Agent

# model_name = 'model_GOOG_50'
# test_stock = 'data/GOOG_2019.csv'
# window_size = 10
# debug = True

# agent = Agent(window_size, pretrained=True, model_name=model_name)

In [37]:
test_stock = 'data/GOOG_2019.csv'
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2019-01-02,1045.849976
1,2019-01-03,1016.059998
2,2019-01-04,1070.709961
3,2019-01-07,1068.390015
4,2019-01-08,1076.280029


In [ ]:
# import logging
# import coloredlogs

# from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
# from trading_bot.methods import evaluate_model

# coloredlogs.install(level='DEBUG')
# switch_k_backend_device()

# test_data = get_stock_data(test_stock)
# initial_offset = test_data[1] - test_data[0]

# test_result, history = evaluate_model(agent, test_data, window_size, debug)
# show_eval_result(model_name, test_result, initial_offset)

In [38]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [ ]:
chart = visualize(df, history, title=test_stock)
chart